In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.stat import Correlation
#import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
#import holidays
from datetime import datetime, timezone
from pyspark.ml.classification import RandomForestClassifier, BinaryLogisticRegressionSummary
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from pyspark.ml.pipeline import PipelineModel

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("US_Accidents") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR") #supress warnings

/opt/conda/lib/python3.7/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/26 13:31:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/26 13:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/26 13:31:31 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# Read in Data
df = spark.read.parquet("updated_dataset.parquet")

In [4]:
# Binary Classification (1 or 2 vs 3 or 4)
#df = df.withColumn('Severity_Binary', when((col("Severity")==1) | (col("Severity")==2), 0).otherwise(1))
df = df.drop('Severity')
df.show(5)

+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|Temperature|Humidity|Pressure|Visibility|Wind_Speed|Precipitation|Weekday|Rush_Hour|Rain|Snow|    SeasonVec|Astronomical_TwilightIndex|Interstate_Indicator|Sex_ratio|Percent_Age_65_over|MedianIncome|Urban_Ratio|Traffic_Interference|Traffic_Intersection|Destination|Percent_Age_15-24|Severity_Binary|
+-----------+--------+--------+----------+----------+-------------+-------+---------+----+----+-------------+--------------------------+--------------------+---------+-------------------+------------+-----------+--------------------+--------------------+-----------+-----------------+---------------+
|         21|      85|      30|         1|        10|            0|      1|        1|   0|   1|(3

In [5]:
# Create list of features
feature_list = []
for col in df.columns:
    if col == 'Severity':
        continue
    elif col == 'Severity_Binary':
        continue
    else:
        feature_list.append(col)

In [6]:
# Split the data into train and test
splits = df.randomSplit([0.8, 0.2], 314)
train = splits[0]
test = splits[1]

In [7]:
# Assemble data for logistic regression model
assembler = VectorAssembler(inputCols=feature_list,
                            outputCol="features")

train = assembler.transform(train)
test = assembler.transform(test)

In [8]:
# Standardize the predictors
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
scalerModel = scaler.fit(train)
scaledTrainData = scalerModel.transform(train)
scaledTestData = scalerModel.transform(test)

In [9]:
# Fit logistic regression model with intercept
from pyspark.ml.classification import LogisticRegression

# instantiate the model
lr = LogisticRegression(labelCol='Severity_Binary',
                        featuresCol='scaledFeatures')

# Fit the model
lrModel = lr.fit(scaledTrainData)
print(f"Detected number of classes: {lrModel.numClasses}")

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

Detected number of classes: 2
Coefficients: DenseMatrix([[-0.13733363,  0.01932292,  0.17555134,  0.04571375,  0.13826136,
               0.00467668, -0.04678153,  0.0051126 ,  0.04897395, -0.01495711,
              -0.31980196, -0.17989337, -0.10271726, -0.0587168 ,  0.5123305 ,
               0.02076322, -0.10572138, -0.01337976,  0.05436457,  0.01042256,
              -0.19156555, -0.14747936, -0.03103486]])
Intercept: [-6.1986898132681745]


In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# compute predictions. this will append column "prediction" to dataframe
lrPred = lrModel.transform(scaledTestData)
lrPred.select("prediction").distinct().show()

# Assuming `lrPred` is your predictions DataFrame
evaluator = MulticlassClassificationEvaluator(
    labelCol="Severity_Binary",
    predictionCol="prediction",
    metricName='truePositiveRateByLabel', 
    metricLabel=1.0
)

recall = evaluator.evaluate(lrPred)
print(f"Recall for label 1.0: {recall}")

accuracy = evaluator.evaluate(lrPred, {evaluator.metricName: "accuracy"})
print(f'Accuracy: {accuracy}')

+----------+
|prediction|
+----------+
|       0.0|
|       1.0|
+----------+



Recall for label 1.0: 0.013864293216194047


Accuracy: 0.8142650243137657


In [11]:
precision = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedPrecision'})
recall = evaluator.evaluate(lrPred, {evaluator.metricName: 'weightedRecall'})
f1_score = evaluator.evaluate(lrPred, {evaluator.metricName: 'f1'})

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1_score}')

Precision: 0.7546567416475418
Recall: 0.8142650243137657
F1 Score: 0.7358098297103297


In [12]:
# Add a column to indicate correct or incorrect predictions
predictions = lrPred.withColumn(
    'is_correct', F.expr("CASE WHEN Severity_Binary = prediction THEN 1 ELSE 0 END")
)

# Calculate accuracy by class
accuracy_by_class = predictions.groupBy('Severity_Binary').agg(
    (F.sum('is_correct') / F.count('Severity_Binary')).alias('accuracy')
)

# Show per-class accuracy
accuracy_by_class.show()

+---------------+--------------------+
|Severity_Binary|            accuracy|
+---------------+--------------------+
|              1|0.013864293216194047|
|              0|  0.9966503683284683|
+---------------+--------------------+



In [13]:
# Calculate metrics by class
labels = [row['Severity_Binary'] for row in lrPred.select('Severity_Binary').distinct().orderBy('Severity_Binary').collect()]

metrics = {}
for label in labels:
    # Filter predictions for the current label
    true_positive = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') == label)).count()
    false_positive = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') == label)).count()
    false_negative = lrPred.filter((F.col('Severity_Binary') == label) & (F.col('prediction') != label)).count()
    true_negative = lrPred.filter((F.col('Severity_Binary') != label) & (F.col('prediction') != label)).count()

    # Precision, Recall, and F1 Score
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    # Store metrics
    metrics[label] = {'precision': precision, 'recall': recall, 'f1_score': f1_score}

# Print metrics for each class
for label, metric in metrics.items():
    print(f"Class {label} - Precision: {metric['precision']:.4f}, Recall: {metric['recall']:.4f}, F1 Score: {metric['f1_score']:.4f}")

Class 0 - Precision: 0.8160, Recall: 0.9967, F1 Score: 0.8973
Class 1 - Precision: 0.4854, Recall: 0.0139, F1 Score: 0.0270
